<a href="https://colab.research.google.com/github/25b3nk/llm-projects/blob/main/chatgpt_assistant_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from openai import OpenAI
from google.colab import userdata
import glob

In [ ]:
openai_api_key = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=openai_api_key)
# client = OpenAI()

In [ ]:
assistant = client.beta.assistants.create(
name="Financial Analyst Assistant",
# instructions="You are an expert financial analyst. Use you knowledge base to answer questions about audited financial statements.",
instructions="You are an helper chat assistant to a company. Use you knowledge base to answer questions about the company and related stuff. If you do not know the answer, tell so.",
model="gpt-4o",
tools=[{"type": "file_search"}],
)

In [ ]:
# Create a vector store caled "Financial Statements"
vector_store = client.beta.vector_stores.create(name="Insure LLM")

# Ready the files for upload to OpenAI
# file_paths = ["edgar/goog-10k.pdf", "edgar/brka-10k.txt"]
file_paths = glob.glob("knowledge-base/**/*.md")

print(file_paths)



['knowledge-base/company/careers.md', 'knowledge-base/company/about.md', 'knowledge-base/company/overview.md', 'knowledge-base/contracts/Contract with Roadway Insurance Inc. for Carllm.md', 'knowledge-base/contracts/Contract with EverGuard Insurance for Rellm.md', 'knowledge-base/contracts/Contract with Stellar Insurance Co. for Rellm.md', 'knowledge-base/contracts/Contract with Pinnacle Insurance Co. for Homellm.md', 'knowledge-base/contracts/Contract with Apex Reinsurance for Rellm.md', 'knowledge-base/contracts/Contract with TechDrive Insurance for Carllm.md', 'knowledge-base/contracts/Contract with Greenstone Insurance for Homellm.md', 'knowledge-base/contracts/Contract with Belvedere Insurance for Markellm.md', 'knowledge-base/contracts/Contract with BrightWay Solutions for Markellm.md', 'knowledge-base/contracts/Contract with GreenField Holdings for Markellm.md', 'knowledge-base/contracts/Contract with GreenValley Insurance for Homellm.md', 'knowledge-base/contracts/Contract with

In [ ]:
# Open the files for reading
file_streams = [open(path, "rb") for path in file_paths]

# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
vector_store_id=vector_store.id, files=file_streams
)

# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=31, failed=0, in_progress=0, total=31)


In [ ]:
# Update assistant to use the new vector store
assistant = client.beta.assistants.update(
assistant_id=assistant.id,
tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [ ]:
assistant

Assistant(id='asst_2dJjaR898B5kazHde1BKtMlD', created_at=1737478068, description=None, instructions='You are an helper chat assistant to a company. Use you knowledge base to answer questions about the company and related stuff. If you do not know the answer, tell so.', metadata={}, model='gpt-4o', name='Financial Analyst Assistant', object='assistant', tools=[FileSearchTool(type='file_search', file_search=FileSearch(max_num_results=None, ranking_options=FileSearchRankingOptions(score_threshold=0.0, ranker='default_2024_08_21')))], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=['vs_FUb0lgqCUDrvKuKYq2i7Spgm'])), top_p=1.0)

In [ ]:
# Upload the user provided file to OpenAI
message_file = client.files.create(
file=open("knowledge-base/employees/Alex Chen.md", "rb"), purpose="assistants"
)

# Create a thread and attach the file to the message
thread = client.beta.threads.create(
messages=[
  {
    "role": "user",
    "content": "When was Alex born ?",
    # Attach the new file to the message.
    "attachments": [
      { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
    ],
  }
]
)

# The thread now has a vector store with that file in its tool resources.
print(thread.tool_resources.file_search)

ToolResourcesFileSearch(vector_store_ids=['vs_WDqAiDiCgrJfSqKNUa4hwuuM'])


In [15]:
# Use the create and poll SDK helper to create a run and poll the status of
# the run until it's in a terminal state.

run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id, assistant_id=assistant.id
)

messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

message_content = messages[0].content[0].text
annotations = message_content.annotations
citations = []
for index, annotation in enumerate(annotations):
  message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
  if file_citation := getattr(annotation, "file_citation", None):
      cited_file = client.files.retrieve(file_citation.file_id)
      citations.append(f"[{index}] {cited_file.filename}")

print(message_content.value)
print("\n".join(citations))

Could you tell me more about Avery's mentions in the documents?



In [21]:
thread = client.beta.threads.create(
messages=[
  {
    "role": "user",
    "content": "What is the name of the company ?",
  }
]
)

In [22]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id, assistant_id=assistant.id
)

messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))
print(len(messages))

1


In [23]:
message_content = messages[0].content[0].text
annotations = message_content.annotations
citations = []
for index, annotation in enumerate(annotations):
  message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
  if file_citation := getattr(annotation, "file_citation", None):
      cited_file = client.files.retrieve(file_citation.file_id)
      citations.append(f"[{index}] {cited_file.filename}")

print(message_content.value)
print("\n".join(citations))

The name of the company is Insurellm[0].
[0] about.md
